In [2]:
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//packages_import.ipynb"    #import all necessary packages - numpy, pandas etc
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//parameters_class.ipynb"                                                                                                  #import the parameters class and the base paramters object
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//simulation_class.ipynb"   
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//outcome_stats_and_plots.ipynb"                                                                                               #import the class that runs the simulation                                                                                            #import the class that runs the simulation                                                                                         #import the class that runs the simulation                                                                                            #import the class that runs the simulation
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//Stable_Ecological_Competition//base_parameters_2.ipynb" import base_parameters
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations/Stable_Ecological_Competition//base_parameters_2.ipynb" import Parameters_for_Analysis

In [5]:
def plot_2d_sensitivity_analysis_initial_pop_vs_evolvability(V_A_scale, V_A2_V_A1_ratios, final_outcomes_matrix):
    color_map = {
        "sp 2 (high-genetic-var) wins, sp 1 (low-genetic-var) extinct": 0,
        "sp 1 (low-genetic-var) wins, sp 2 (high-genetic-var) extinct": 1,
        "both coexist": 2,
        "both extinct": 3
    }

    # Convert the final outcomes to numerical values
    numerical_matrix = np.vectorize(color_map.get)(final_outcomes_matrix)   

    # Create the heatmap
    heatmap = go.Heatmap(
        z=numerical_matrix.T,  # Transpose the matrix to match the correct axis
        x=V_A_scale,
        y=V_A2_V_A1_ratios,
        # colorscale=[[0/3, 'red'], [1/3, 'blue'], [2/3, 'green'], [3/3, 'black']],
        colorscale=[
            [0.0, 'red'],
            [0.25, 'red'],
            [0.25, 'blue'],
            [0.5, 'blue'],
            [0.5, 'green'],
            [0.75, 'green'],
            [0.75, 'black'],
            [1.0, 'black']
        ],
        zmin=0,
        zmax=3,
        showscale = False,
        colorbar=dict(
            tickvals=[0, 1, 2, 3],
            ticktext=list(color_map.keys()),
        )
    )

    fig = go.Figure(data=[heatmap])

    # legend_labels = [
    #     ("Slow evolver extinct", 'red'),
    #     ("Fast evolver extinct", 'blue'),
    #     ('Both coexist', 'green'),
    #     ('Both extinct', 'black')
    # ]

    # for label, color in legend_labels:
    #     fig.add_trace(go.Scatter(
    #         x=[None], y=[None], mode='markers',
    #         marker=dict(color=color),
    #         showlegend=True,
    #         name=label
    #     ))



    fig.update_layout(
        xaxis=dict(
            title='Base Rate of Evolution (log)',
            type='log',
            tickvals=[0.001, 0.01, 0.1],
            ticktext=["0.001", "0.01", "0.1"],
            title_font=dict(size=45, color='black'),
            tickfont=dict(size=30)  # Increase ticktext font size
        ),
        yaxis=dict(
            title='Evolvability Ratio [fast-ev/slow-ev] (log)',
            type='log',
            tickvals=[1, 10, 100, 1000],
            ticktext=['1', '10', '100', '1000'],
            title_font=dict(size=40, color='black'),
            tickfont=dict(size=30)  # Increase ticktext font size
        ),
        template='plotly',
        width=1000,
        height=800,
    )
    

    pio.show(fig)

def generate_parameters_with_varied_parameters(V_A_scale, V_A2_V_A1_ratios):
    parameters_list = []
    for V_A_value in V_A_scale:
        for V_A2_V_A1 in V_A2_V_A1_ratios:
            parameters_dict = Parameters_for_Analysis(base_parameters=base_parameters, V_A1= V_A_value, V_A2=V_A_value).__dict__.copy()
            parameters_dict['V_A2'] = parameters_dict['V_A1'] * V_A2_V_A1
            parameters = Parameters_for_Analysis(base_parameters=base_parameters, **parameters_dict)
            parameters_list.append(parameters)

            
    return parameters_list

def run_sensitivity_analysis(parameters_list):
    final_outcomes_list = []
    for parameters in parameters_list:
        final_outcome_instance = run_simulation_and_get_outcome_instance(parameters)
        final_outcomes_list.append(final_outcome_instance.outcome)
    return final_outcomes_list

def main():
    V_A_value_powers = np.arange(-3, 0, 0.0625)
    V_A2_V_A1_powers = np.arange(0, 3, 0.0625)
    
    V_A_scale = np.power(10.0, V_A_value_powers)
    V_A2_V_A1_ratios = np.power(10.0, V_A2_V_A1_powers)
    
    parameters_list = generate_parameters_with_varied_parameters(V_A_scale, V_A2_V_A1_ratios)
    final_outcomes_list = run_sensitivity_analysis(parameters_list)
    
    final_outcomes_matrix = np.array(final_outcomes_list).reshape(len(V_A_scale), len(V_A2_V_A1_ratios))
    
    plot_2d_sensitivity_analysis_initial_pop_vs_evolvability(V_A_scale, V_A2_V_A1_ratios, final_outcomes_matrix)

if __name__ == "__main__":
    main()

